# LLM Project - Complete Tutorial

This notebook provides a comprehensive tutorial on using T5 models, based on `solution-01-t5.ipynb`.

**What you'll learn:**
- Tokenization: Converting text to numbers and back
- Model loading: Understanding T5 architecture
- Text generation: How models create output
- Embeddings visualization: PCA and cosine similarity
- Forward passes: Encoder-decoder mechanics

**Prerequisites:** None! This notebook installs everything you need.


In [1]:
## Step 1: Setup and Installation

First, we'll clone the repository and install all dependencies.

SyntaxError: unterminated string literal (detected at line 3) (3212902300.py, line 3)

In [ ]:
# Clone the repository
!git clone https://github.com/SabraHashemi/llm-project.git
%cd llm-project


Install required packages for transformers, visualization, and machine learning.


In [ ]:
%pip install -q transformers torch matplotlib scikit-learn numpy python-dateutil
print("✅ Dependencies installed!")


In [ ]:
## Step 2: Import Modules

Import the necessary libraries and our custom tokenizer/model loader modules.


ModuleNotFoundError: No module named 'llm_tokenizers'

In [ ]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity

# Add project root to Python path
sys.path.insert(0, '.')

# Import our custom modules
from llm_tokenizers import BaseTokenizerWrapper
from llm_models import Seq2SeqModelLoader

print("✅ All modules imported successfully!")


In [ ]:
## Step 3: Initialize Tokenizer and Model

Load the T5-small tokenizer and model. This will download the models on first run (~240MB).


In [ ]:
# Initialize tokenizer
print("Loading tokenizer...")
tokenizer = BaseTokenizerWrapper("t5-small")
print(f"✅ Tokenizer loaded! Vocabulary size: {tokenizer.vocab_size:,} tokens")

# Initialize model
print("\nLoading model (this may take a moment on first run)...")
model = Seq2SeqModelLoader("t5-small")
print(f"✅ Model loaded!")
print(f"   - Hidden size: {model.hidden_size}")
print(f"   - Number of layers: {model.num_layers}")
print(f"   - Number of attention heads: {model.num_heads}")


## Step 4: Encoding and Decoding

**What is tokenization?**
- **Encoding**: Converts human-readable text → token IDs (numbers)
- **Decoding**: Converts token IDs (numbers) → human-readable text

This is the fundamental conversion between text and the numerical representation models use.


In [ ]:
# Example: Encode text to token IDs
sentence = "hello, this is a sentence!"
tokens = tokenizer.encode(sentence)

print(f"Original text: '{sentence}'")
print(f"Token IDs: {tokens['input_ids']}")
print(f"Attention mask: {tokens['attention_mask']}")
print(f"\nDecoded back: '{tokenizer.decode(tokens['input_ids'])}'")
print("\n💡 Note: The tokenizer automatically added </s> (end-of-sequence token)")


### Special Tokens

Each model uses special tokens with specific meanings:
- **EOS** (`</s>`): End of sequence
- **PAD** (`<pad>`): Padding token (also used as decoder start in T5)
- **BOS**: Beginning of sequence (T5 doesn't use this)


In [ ]:
# Get special tokens
special = tokenizer.get_special_tokens()
print("Special tokens for T5:")
print(f"  EOS token: '{special['eos_token']}' (ID: {special['eos_token_id']})")
print(f"  PAD token: '{special['pad_token']}' (ID: {special['pad_token_id']})")
print(f"  BOS token: {special['bos_token']} (T5 doesn't use BOS)")


## Step 5: Forward Pass (Encoder-Decoder Architecture)

**How T5 works:**
1. **Encoder**: Processes the input text (e.g., "translate english to german: hello")
2. **Decoder**: Generates output tokens one by one
3. **First step**: Decoder starts with `<pad>` token (T5's special start token)
4. **Output**: Model produces logits (probabilities) for each possible next token


In [ ]:
# Prepare input
input_sentence = "translate english to german: hello, how are you?"
tokens = tokenizer.encode(input_sentence, return_tensors="pt")
decoder_input_ids = torch.tensor([[tokenizer.tokenizer.pad_token_id]])

print(f"Input: '{input_sentence}'")
print(f"Encoder input shape: {tokens['input_ids'].shape}")
print(f"Decoder input (starting token): {decoder_input_ids.tolist()}")

# Forward pass
with torch.no_grad():
    output = model(**tokens, decoder_input_ids=decoder_input_ids)

print(f"\n✅ Forward pass completed!")
print(f"Output logits shape: {output.logits.shape}")
print(f"   → Shape means: [batch=1, sequence=1, vocab={model.vocab_size:,}]")
print(f"   → For each position, we have {model.vocab_size:,} logits (one per token)")
print(f"\nOutput contains:")
print(f"  - logits: Probabilities for next token")
print(f"  - past_key_values: Cached attention states")
print(f"  - encoder_last_hidden_state: Final encoder representation")


## Step 6: Text Generation

**Autoregressive Generation:**
Models generate text token by token. Each new token depends on all previous tokens.

**Greedy Decoding:** Always picks the most likely next token (highest probability).

The `model.generate()` method does this automatically with optimizations!


In [ ]:
# Generate text using model.generate()
# Note: We need to prepare tokens again for generation
input_sentence = "translate english to german: hello, how are you?"
tokens = tokenizer.encode(input_sentence, return_tensors="pt")

with torch.no_grad():
    generated_ids = model.generate(**tokens, max_length=20)

generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print(f"Input:  '{input_sentence}'")
print(f"Output: '{generated_text}'")
print(f"\n💡 The model translated English to German!")
print(f"💡 model.generate() is optimized and faster than manual generation")


## Step 7: Token Embeddings Visualization

**What are embeddings?**
- Each word/token is represented as a high-dimensional vector (512 dimensions for T5-small)
- These vectors capture semantic meaning
- Similar words have similar embeddings

**Visualization techniques:**
1. **PCA (Principal Component Analysis)**: Reduces 512D → 2D for visualization
2. **Cosine Similarity**: Measures how "similar" two word embeddings are (0-1 scale)


In [ ]:
# Words to visualize
words = [
    "chair",
    "table",
    "plate",
    "knife",
    "spoon",
    "horse",
    "goat",
    "sheep",
    "cat",
    "dog",
]

print(f"📝 Analyzing embeddings for {len(words)} words:")
for i, word in enumerate(words, 1):
    print(f"   {i}. {word}")

# Get token IDs for first token of each word
word_tokens = tokenizer.encode(words, return_tensors="pt", padding=True)["input_ids"][:, 0]
print(f"\nToken IDs: {word_tokens.tolist()}")

# Extract embeddings from the model's shared embedding layer
with torch.no_grad():
    token_embeddings = model.model.shared(word_tokens).cpu().detach().numpy()

print(f"✅ Embeddings extracted!")
print(f"   Shape: {token_embeddings.shape}")
print(f"   → Each word is a {token_embeddings.shape[1]}-dimensional vector")


### PCA Visualization (2D Projection)

PCA reduces high-dimensional embeddings to 2D so we can visualize them. Words that are semantically similar should appear close together!


In [ ]:
# Apply PCA to reduce dimensions from 512D to 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(token_embeddings)

print(f"Explained variance per component: {pca.explained_variance_ratio_}")
print(f"Total variance explained: {sum(pca.explained_variance_ratio_):.2%}")
print("→ This shows how much information is preserved in 2D")

# Create PCA plot
plt.figure(figsize=(12, 8))
plt.scatter(X_pca[:, 0], X_pca[:, 1], s=200, alpha=0.7, edgecolors='black', linewidth=1.5)

# Add labels with better positioning
for i, word in enumerate(words):
    plt.annotate(word, (X_pca[i, 0], X_pca[i, 1]), 
                xytext=(5, 5), textcoords='offset points', 
                fontsize=11, fontweight='bold')

plt.xlabel('First Principal Component', fontsize=13)
plt.ylabel('Second Principal Component', fontsize=13)
plt.title('Token Embeddings - PCA Visualization (2D Projection)\nWords close together are semantically similar', 
          fontsize=15, fontweight='bold')
plt.grid(True, alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("   - Furniture words (chair, table, plate, knife, spoon) should cluster together")
print("   - Animal words (horse, goat, sheep, cat, dog) should cluster together")
print("   - This shows the model learned semantic relationships!")


### Cosine Similarity Matrix

Cosine similarity measures how similar two word embeddings are:
- **1.0** = Identical (same word)
- **0.8-1.0** = Very similar (e.g., "cat" and "dog")
- **0.5-0.8** = Somewhat related
- **0.0-0.5** = Unrelated
- **Negative** = Opposites or very different


In [ ]:
# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(token_embeddings)

print(f"Similarity matrix shape: {similarity_matrix.shape}")
print(f"→ Shows pairwise similarity between all {len(words)} words\n")

# Create heatmap with better styling
fig, ax = plt.subplots(figsize=(12, 10))
cax = ax.imshow(similarity_matrix, cmap="RdYlBu_r", vmin=-1, vmax=1, aspect='auto')
cbar = fig.colorbar(cax, ax=ax, label='Cosine Similarity', shrink=0.8)
cbar.ax.tick_params(labelsize=11)

ax.set_xticks(range(len(words)))
ax.set_yticks(range(len(words)))
ax.set_xticklabels(words, rotation=45, ha='right', fontsize=11)
ax.set_yticklabels(words, fontsize=11)
ax.set_title('Cosine Similarity Matrix\n(Red = similar, Blue = different)', 
             fontsize=15, fontweight='bold', pad=20)

# Add similarity values to the plot (only show if similarity > 0.3 for readability)
for i in range(len(words)):
    for j in range(len(words)):
        sim_val = similarity_matrix[i, j]
        # Use white text for dark backgrounds, black for light
        text_color = 'white' if abs(sim_val) > 0.5 else 'black'
        if abs(sim_val) > 0.3 or i == j:  # Show diagonal and significant similarities
            ax.text(j, i, f'{sim_val:.2f}',
                   ha="center", va="center", 
                   color=text_color, fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

# Print most similar pairs
print("\n💡 Most similar word pairs:")
similarities = []
for i in range(len(words)):
    for j in range(i + 1, len(words)):
        similarities.append((words[i], words[j], similarity_matrix[i, j]))

similarities.sort(key=lambda x: x[2], reverse=True)
for i, (word1, word2, sim) in enumerate(similarities[:5], 1):
    print(f"   {i}. '{word1}' ↔ '{word2}': {sim:.3f}")


## Summary

**What you learned:**
1. ✅ **Tokenization**: Converting text to numbers (token IDs)
2. ✅ **Model Architecture**: Understanding encoder-decoder structure
3. ✅ **Forward Pass**: How models process input and generate output
4. ✅ **Text Generation**: Autoregressive token-by-token generation
5. ✅ **Embeddings**: How words are represented as vectors
6. ✅ **Visualization**: PCA and cosine similarity for understanding embeddings

**Next steps:**
- Try different input prompts
- Experiment with different T5 model sizes (t5-base, t5-large)
- Explore attention mechanisms in `solution-02-attention.ipynb`
- Modify the code to experiment with different models (BERT, GPT-2, etc.)

**Resources:**
- Full tutorial: `labs/solution-01-t5.ipynb`
- Test suite: `test_t5_notebook.py`
- More examples: `examples/` folder
